<a href="https://colab.research.google.com/github/HromovAndrey/II_Homework_mood_3_part_3/blob/main/%D0%94%D0%97_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [42]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module2/data/ship_passengers.csv",
                 index_col="Unnamed: 0")

In [43]:
df.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [44]:
df = df[['pclass', 'age', 'embarked', 'fare']]
df.head()

,pclass,age,embarked,fare
0,3,22.0,S,7.2500
1,1,38.0,C,71.2833
2,3,26.0,S,7.9250
3,1,35.0,S,53.1000
4,3,35.0,S,8.0500


# Завдання 1
Очистіть дані від викидів(лише `fare`), розділіть на тренувальну та тестову чатини

In [45]:
df.describe()

,pclass,age,fare
count,891.000000,714.000000,891.000000
mean,2.308642,29.699118,32.204208
std,0.836071,14.526497,49.693429
min,1.000000,0.420000,0.000000
25%,2.000000,20.125000,7.910400
50%,3.000000,28.000000,14.454200
75%,3.000000,38.000000,31.000000
max,3.000000,80.000000,512.329200


In [46]:
Q1 = df['fare'].quantile(0.25)
Q3 = df['fare'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['fare'] >= (Q1 - 1.5 * IQR)) & (df['fare'] <= (Q3 + 1.5 * IQR))]

In [47]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [48]:
train_df.head()
test_df.head()

,pclass,age,embarked,fare
493,1,71.0,C,49.5042
821,3,27.0,S,8.6625
381,3,1.0,C,15.7417
881,3,33.0,S,7.8958
420,3,NaN,C,7.8958


# Завдання 2
Створіть Pipeline для обробки даних

In [59]:
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [60]:
categorical_features = ['pclass', 'embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [61]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print(preprocessor)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['age', 'fare']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pclass', 'embarked'])])


# Завдання 3
Створіть остаточну модель та натренуйте її

In [62]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

In [63]:
X_train = train_df.drop('fare', axis=1)
y_train = train_df['fare']
X_test = test_df.drop('fare', axis=1)
y_test = test_df['fare']

In [64]:
print("Стовпці у тренувальній вибірці:", X_train.columns)
print("Стовпці у тестовій вибірці:", X_test.columns)

Стовпці у тренувальній вибірці: Index(['pclass', 'age', 'embarked'], dtype='object')
Стовпці у тестовій вибірці: Index(['pclass', 'age', 'embarked'], dtype='object')


In [65]:
model.fit(X_train, y_train)

ValueError: A given column is not a column of the dataframe

# Завдання 4
Виведіть основні метрики моделі

# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

Попробуйте:
* не видаляти викиди
* змінити обробку даних(imputer та кодування категоріальних даних)
* змінити параметри дерева(глибина, кількість точок у листках тощо

# Завдання 6
Збережіть модель

In [66]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Завантажимо дані
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module2/data/ship_passengers.csv", index_col="Unnamed: 0")

# Оберемо потрібні стовпчики
df = df[['pclass', 'age', 'embarked', 'fare']]

# Виведемо основні числові характеристики
print(df.describe())

# Видалимо викиди у fare
Q1 = df['fare'].quantile(0.25)
Q3 = df['fare'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['fare'] >= (Q1 - 1.5 * IQR)) & (df['fare'] <= (Q3 + 1.5 * IQR))]

# Розділимо дані на тренувальну та тестову частини
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print("Тренувальна вибірка:")
print(train_df.head())
print("Тестова вибірка:")
print(test_df.head())


           pclass         age        fare
count  891.000000  714.000000  891.000000
mean     2.308642   29.699118   32.204208
std      0.836071   14.526497   49.693429
min      1.000000    0.420000    0.000000
25%      2.000000   20.125000    7.910400
50%      3.000000   28.000000   14.454200
75%      3.000000   38.000000   31.000000
max      3.000000   80.000000  512.329200
Тренувальна вибірка:
     pclass   age embarked     fare
147       3   9.0        S  34.3750
868       3   NaN        S   9.5000
60        3  22.0        C   7.2292
468       3   NaN        Q   7.7250
777       3   5.0        S  12.4750
Тестова вибірка:
     pclass   age embarked     fare
493       1  71.0        C  49.5042
821       3  27.0        S   8.6625
381       3   1.0        C  15.7417
881       3  33.0        S   7.8958
420       3   NaN        C   7.8958


In [67]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Pipeline для числових стовпчиків
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# Pipeline для категоріальних стовпчиків
categorical_features = ['pclass', 'embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Об'єднуємо Pipeline у ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print(preprocessor)


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['age', 'fare']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pclass', 'embarked'])])


In [68]:
from sklearn.linear_model import LinearRegression

# Створюємо остаточну модель, яка включає попередню обробку даних та модель
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

# Розділимо дані на X та y
X_train = train_df.drop('fare', axis=1)
y_train = train_df['fare']
X_test = test_df.drop('fare', axis=1)
y_test = test_df['fare']

# Перевіримо наявність стовпця 'fare' у тренувальній та тестовій вибірках
print("Стовпці у тренувальній вибірці:", X_train.columns)
print("Стовпці у тестовій вибірці:", X_test.columns)

# Навчимо модель
model.fit(X_train, y_train)


Стовпці у тренувальній вибірці: Index(['pclass', 'age', 'embarked'], dtype='object')
Стовпці у тестовій вибірці: Index(['pclass', 'age', 'embarked'], dtype='object')


ValueError: A given column is not a column of the dataframe